In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from lib.utils.config_parse import cfg_from_file
# from lib.ssds_train import test_model

from lib.dataset.dataset_factory import load_data
from lib.utils.config_parse import cfg
from lib.modeling.model_builder import create_model

from lib.layers import Detect
from lib.utils.timer import Timer

import pickle
import numpy as np
import copy

import sys
sys.path.append('/workspace/raid/data/jgusak/for_yulia/')

from torchvision.models import resnet18


## Prepare ResNet18 

In [ ]:
m = resnet18(pretrained = True)
checkpoint = copy.deepcopy(m.state_dict())

# change the name of the weights which exists in other model
change_dict = {
        'conv1.weight':'base.0.weight',
        'bn1.running_mean':'base.1.running_mean',
        'bn1.running_var':'base.1.running_var',
        'bn1.bias':'base.1.bias',
        'bn1.weight':'base.1.weight',
        }

for k, v in list(checkpoint.items()):
    for _k, _v in list(change_dict.items()):
        if _k == k:
            new_key = k.replace(_k, _v)
            checkpoint[new_key] = checkpoint.pop(k)
            
change_dict = {'layer1.{:d}.'.format(i):'base.{:d}.'.format(i+4) for i in range(20)}
change_dict.update({'layer2.{:d}.'.format(i):'base.{:d}.'.format(i+6) for i in range(20)})
change_dict.update({'layer3.{:d}.'.format(i):'base.{:d}.'.format(i+8) for i in range(30)})

for k, v in list(checkpoint.items()):
    for _k, _v in list(change_dict.items()):
        if _k in k:
            new_key = k.replace(_k, _v)
            checkpoint[new_key] = checkpoint.pop(k)

In [7]:
checkpoint.keys()

odict_keys(['bn1.num_batches_tracked', 'layer4.0.conv1.weight', 'layer4.0.bn1.weight', 'layer4.0.bn1.bias', 'layer4.0.bn1.running_mean', 'layer4.0.bn1.running_var', 'layer4.0.bn1.num_batches_tracked', 'layer4.0.conv2.weight', 'layer4.0.bn2.weight', 'layer4.0.bn2.bias', 'layer4.0.bn2.running_mean', 'layer4.0.bn2.running_var', 'layer4.0.bn2.num_batches_tracked', 'layer4.0.downsample.0.weight', 'layer4.0.downsample.1.weight', 'layer4.0.downsample.1.bias', 'layer4.0.downsample.1.running_mean', 'layer4.0.downsample.1.running_var', 'layer4.0.downsample.1.num_batches_tracked', 'layer4.1.conv1.weight', 'layer4.1.bn1.weight', 'layer4.1.bn1.bias', 'layer4.1.bn1.running_mean', 'layer4.1.bn1.running_var', 'layer4.1.bn1.num_batches_tracked', 'layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'layer4.1.bn2.running_mean', 'layer4.1.bn2.running_var', 'layer4.1.bn2.num_batches_tracked', 'fc.weight', 'fc.bias', 'base.0.weight', 'base.1.weight', 'base.1.bias', 'base.1.running_mean', 'ba

In [10]:
config_file =  './experiments/cfgs/ssd_resnet18_train_voc.yml'
cfg_from_file(config_file)

print(config_file)


# Build model
print('===> Building model')
model, _ = create_model(cfg.MODEL)



./experiments/cfgs/ssd_resnet18_train_voc.yml
===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]


/workspace/home/jgusak/ssds.pytorch/lib/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [12]:
model_dict = model.state_dict()
pretrained_dict = {k:v for k,v in checkpoint.items() if k in model_dict.keys()}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [13]:
model_dict.keys()

odict_keys(['base.0.weight', 'base.1.weight', 'base.1.bias', 'base.1.running_mean', 'base.1.running_var', 'base.1.num_batches_tracked', 'base.4.conv1.weight', 'base.4.bn1.weight', 'base.4.bn1.bias', 'base.4.bn1.running_mean', 'base.4.bn1.running_var', 'base.4.bn1.num_batches_tracked', 'base.4.conv2.weight', 'base.4.bn2.weight', 'base.4.bn2.bias', 'base.4.bn2.running_mean', 'base.4.bn2.running_var', 'base.4.bn2.num_batches_tracked', 'base.5.conv1.weight', 'base.5.bn1.weight', 'base.5.bn1.bias', 'base.5.bn1.running_mean', 'base.5.bn1.running_var', 'base.5.bn1.num_batches_tracked', 'base.5.conv2.weight', 'base.5.bn2.weight', 'base.5.bn2.bias', 'base.5.bn2.running_mean', 'base.5.bn2.running_var', 'base.5.bn2.num_batches_tracked', 'base.6.conv1.weight', 'base.6.bn1.weight', 'base.6.bn1.bias', 'base.6.bn1.running_mean', 'base.6.bn1.running_var', 'base.6.bn1.num_batches_tracked', 'base.6.conv2.weight', 'base.6.bn2.weight', 'base.6.bn2.bias', 'base.6.bn2.running_mean', 'base.6.bn2.running_var'

## Prepare ResNet18 compressed

In [7]:
path2pretrained_imagenet = "/workspace/raid/data/jgusak/for_yulia/resnet18_cp3_compressed_loss1.260267.pth"

cm = torch.load(path2pretrained_imagenet).module
dict(cm.named_children()).keys()

checkpoint = copy.deepcopy(cm.state_dict())

In [8]:
dict(cm.named_children()).keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [10]:
# change the name of the weights which exists in other model
change_dict = {
        'bn1.running_mean':'base.1.running_mean',
        'bn1.running_var':'base.1.running_var',
        'bn1.bias':'base.1.bias',
        'bn1.weight':'base.1.weight',
        }

for k, v in list(checkpoint.items()):
    for _k, _v in list(change_dict.items()):
        if _k == k:
            new_key = k.replace(_k, _v)
            checkpoint[new_key] = checkpoint.pop(k)
            
            
change_dict = {'layer1.{:d}.'.format(i):'base.{:d}.'.format(i+4) for i in range(20)}
change_dict.update({'layer2.{:d}.'.format(i):'base.{:d}.'.format(i+6) for i in range(20)})
change_dict.update({'layer3.{:d}.'.format(i):'base.{:d}.'.format(i+8) for i in range(30)})

for k, v in list(checkpoint.items()):
    for _k, _v in list(change_dict.items()):
        if _k in k:
            new_key = k.replace(_k, _v)
            checkpoint[new_key] = checkpoint.pop(k)

In [11]:
config_file =  './experiments/cfgs/ssd_resnet18_train_voc.yml'
cfg_from_file(config_file)

print(config_file)


# Build model
print('===> Building model')
model, priorbox = create_model(cfg.MODEL)

./experiments/cfgs/ssd_resnet18_train_voc.yml
===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]


/workspace/home/jgusak/ssds.pytorch/lib/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [12]:
model.base[0] = copy.deepcopy(cm.conv1)

In [13]:
model_dict = model.state_dict()
pretrained_dict = {k:v for k,v in checkpoint.items() if k in model_dict.keys()}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Run

In [7]:
 # Load data
print('===> Loading data')
train_loader = load_data(cfg.DATASET, 'train') if 'train' in cfg.PHASE else None
eval_loader = load_data(cfg.DATASET, 'eval') if 'eval' in cfg.PHASE else None
test_loader = load_data(cfg.DATASET, 'test') if 'test' in cfg.PHASE else None

===> Loading data


In [8]:
with torch.no_grad():
    priors = Variable(priorbox.forward())
    detector = Detect(cfg.POST_PROCESS, priors)

In [9]:
# Utilize GPUs for computation
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('Utilize GPUs for computation')
    print('Number of GPU available', torch.cuda.device_count())
    model.cuda()
    priors.cuda()
    cudnn.benchmark = True

Utilize GPUs for computation
Number of GPU available 8


In [10]:
output_dir = cfg.EXP_DIR
checkpoint = cfg.RESUME_CHECKPOINT
checkpoint_prefix = cfg.CHECKPOINTS_PREFIX

checkpoint, checkpoint_prefix

('', 'ssd_resnet_18_voc')

In [11]:
def test_epoch(model, data_loader, detector, output_dir, use_gpu):
        model.eval()

        dataset = data_loader.dataset
        num_images = len(dataset)
        num_classes = detector.num_classes
        all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]
        empty_array = np.transpose(np.array([[],[],[],[],[]]),(1,0))

        _t = Timer()

        with torch.no_grad():
            for i in iter(range((num_images))):
                img = dataset.pull_image(i)
                scale = [img.shape[1], img.shape[0], img.shape[1], img.shape[0]]
                if use_gpu:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0).cuda())
                else:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0))
                _t.tic()
                # forward
                out = model(images, phase='eval')

                # detect
                detections = detector.forward(out)

                time = _t.toc()

                # TODO: make it smart:
                for j in range(1, num_classes):
                    cls_dets = list()
                    for det in detections[0][j]:
                        if det[0] > 0:
                            d = det.cpu().numpy()
                            score, box = d[0], d[1:]
                            box *= scale
                            box = np.append(box, score)
                            cls_dets.append(box)
                    if len(cls_dets) == 0:
                        cls_dets = empty_array
                    all_boxes[j][i] = np.array(cls_dets)

                # log per iter
                log = '\r==>Test: || {iters:d}/{epoch_size:d} in {time:.3f}s [{prograss}]\r'.format(
                        prograss='#'*int(round(10*i/num_images)) + '-'*int(round(10*(1-i/num_images))), iters=i, epoch_size=num_images,
                        time=time)
                sys.stdout.write(log)
                sys.stdout.flush()

        # write result to pkl
        with open(os.path.join(output_dir, 'detections.pkl'), 'wb') as f:
            pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

        # currently the COCO dataset do not return the mean ap or ap 0.5:0.95 values
        print('Evaluating detections')
        data_loader.dataset.evaluate_detections(all_boxes, output_dir)

In [12]:
test_epoch(model, test_loader, detector, output_dir+'_compressed', use_gpu)

KeyboardInterrupt: 

In [ ]:
model